In [1]:
%%capture
%pip install psycopg2-binary pandas

In [19]:
import psycopg2
import pandas as pd
import warnings
from datetime import datetime
from io import StringIO

warnings.filterwarnings("ignore", category=UserWarning, message="pandas only supports SQLAlchemy")

In [11]:
conn = psycopg2.connect(**{
    "host": "54.174.251.9",
    "port": 5432,
    "user": "postgres",
    "password": "postgres",
    "database": "autoprovision",
})
cur = conn.cursor()

In [12]:
list_tables_query = """
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_type = 'BASE TABLE' AND table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_schema, table_name;
"""
cur.execute(list_tables_query)

tables = cur.fetchall()

In [17]:
today = datetime.now().strftime('%Y-%m-%d')

In [13]:
for schema, table in tables:
	print(f"{schema}.{table}")
	query = f"SELECT * FROM {schema}.{table};"
	df = pd.read_sql_query(query, conn)
	print(f"\nTabela {schema}.{table} carregada em um DataFrame:")
	print(df.head(3))
	s3_path = f"autoprovision/zoho-crm/{today}/{schema}__{table}.csv"

	csv_buffer = StringIO()
	df.to_csv(csv_buffer, index=False)

	# s3_client.put_object(
	# 	Bucket='nome-do-seu-bucket',
	# 	Key=s3_path,
	# 	Body=csv_buffer.getvalue()
	# )

crm.contrato

Tabela crm.contrato carregada em um DataFrame:
                                     id                          id_clicksign  \
0  1cd55428-6c72-4ff1-9088-dd22aee3bfb9  acf581bf-50b6-47c2-bfc6-d0bd9e58cc82   
1  e97b70b7-2258-4506-85e5-1621e7534af7  135a96b5-5bb9-4e61-8279-4cdee7cc4f5e   
2  69cca094-60be-4eba-b520-b26d15f7e678  8fa7669f-e0a6-40ec-8ac5-85f855874341   

                                       url_documento data_geracao_contrato  \
0  https://sandbox.clicksign.com/accounts/11860/f...            2024-11-05   
1  https://sandbox.clicksign.com/accounts/11860/f...            2024-11-05   
2  https://sandbox.clicksign.com/accounts/11860/f...            2024-11-05   

                                            template      horario_assinatura  
0  {"document": {"path": "/Contratos/Leandro Cost...                     NaT  
1  {"document": {"path": "/Contratos/Joaquim Gime... 2024-11-05 00:36:46.578  
2  {"document": {"path": "/Contratos/Joaquim Gime... 2024-11-05 